In [4]:
import sys
from pathlib import Path
import shutil
import json
import csv
import re

from shapely.geometry import Point
import requests

DIR = Path('..')
sys.path.append(str(DIR))

import wavetrace as wt

%load_ext autoreload
%autoreload 2



The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
3601

0.0004166666666666667

In [ ]:
# Test

# in_path = (DIR/'tests'/'data'/'srtm3'/'S37E175.hgt').resolve()
in_path = (DIR/'tests'/'data'/'srtm3'/'S36E174.SRTMGL3.hgt.zip').resolve()
out_path = DIR/'tests'/'data'/'tmp'/'sample_satellite_shadow.tif'
if not out_path.exists():
    out_path.touch()
out_path = out_path.resolve()
tid = wt.extract_tile_id(in_path)
wt.compute_satellite_shadow(in_path, 152, out_path)

In [ ]:
# # Visually check that SRTM tiles cover NZ by 
# # making polygons and viewing them at https://geojson.io.

# tids = wt.SRTM_NZ_TILE_IDS
# collection = {
#     'type': 'FeatureCollection', 
#     'features': list(map(wt.build_feature, tids))
#     }
# json.dumps(collection)

In [ ]:
# Initialize

TMP = DIR/'tmp'
wt.rm_paths(TMP)
#transmitters_path = DIR/'tests'/'data'/'transmitters.csv'
transmitters_path = DIR/'tests'/'data'/'transmitters_single.csv'
api_key = wt.get_secret('GITLAB_API_KEY') # Need this to download topo
high_definition = False
if high_definition:
    topography_path = TMP/'srtm1'
else:
    topography_path = TMP/'srtm3'


In [ ]:
# Process transmitters

out_path = TMP/'splat_files'   
wt.process_transmitters(transmitters_path, out_path)

% ls {out_path}

In [ ]:
# Download topography

transmitters = wt.read_transmitters(transmitters_path)
print('transmitters=', transmitters)

tids = wt.compute_tile_ids(transmitters)
print('Tiles to download =', tids)

wt.download_topography(tids, topography_path, 
  high_definition=high_definition, api_key=api_key)

% ls {path}

In [ ]:
# Process topography

out_path = TMP/'splat_files'
wt.process_topography(topography_path, out_path, 
  high_definition=high_definition)

% ls {out_path}

In [ ]:
# Compute coverage

in_path = TMP/'splat_files'
out_path = TMP/'coverage_reports'
wt.compute_coverage(in_path, out_path, high_definition=high_definition)

% ls {out_path}

In [ ]:
## Clean up

# wt.rm_paths(TMP)